# Module Import

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets

# Device

In [2]:
if torch.cuda.is_available() :
    DEVICE = torch.device('cuda')
else : 
    DEVICE = torch.device('cpu')
print('Using PyTorch version : ',torch.__version__ ,"Device : ",DEVICE )

Using PyTorch version :  1.11.0 Device :  cpu


# Parameter Set

In [4]:
BATCH_SIZE = 32
EPOCHS = 10

# Data Load

In [10]:
data_transforms = {
    'train' : transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
    ]),
    'val' : transforms.Compose([
        transforms.CenterCrop(224),
        transforms.Resize(256),
        transforms.ToTensor(),
        transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
    ])
}

image_datasets = { x : datasets.ImageFolder("dogs-vs-cats", data_transforms[x]) for x in ['train','val'] 
}
dataloaders = {x : torch.utils.data.DataLoader(image_datasets[x],
batch_size = BATCH_SIZE,
num_workers= 0,
shuffle = True) for x in ['train','val']}

# Check Data

In [11]:
for (X_train, y_train) in dataloaders['train']:
    print('X_train:',X_train.size(),'type:', X_train.type())
    print('y_train:',y_train.size(),'type:', y_train.type())

X_train: torch.Size([32, 3, 224, 224]) type: torch.FloatTensor
y_train: torch.Size([32]) type: torch.LongTensor
X_train: torch.Size([32, 3, 224, 224]) type: torch.FloatTensor
y_train: torch.Size([32]) type: torch.LongTensor
X_train: torch.Size([32, 3, 224, 224]) type: torch.FloatTensor
y_train: torch.Size([32]) type: torch.LongTensor
X_train: torch.Size([32, 3, 224, 224]) type: torch.FloatTensor
y_train: torch.Size([32]) type: torch.LongTensor
X_train: torch.Size([32, 3, 224, 224]) type: torch.FloatTensor
y_train: torch.Size([32]) type: torch.LongTensor
X_train: torch.Size([32, 3, 224, 224]) type: torch.FloatTensor
y_train: torch.Size([32]) type: torch.LongTensor
X_train: torch.Size([32, 3, 224, 224]) type: torch.FloatTensor
y_train: torch.Size([32]) type: torch.LongTensor
X_train: torch.Size([32, 3, 224, 224]) type: torch.FloatTensor
y_train: torch.Size([32]) type: torch.LongTensor
X_train: torch.Size([32, 3, 224, 224]) type: torch.FloatTensor
y_train: torch.Size([32]) type: torch.Lon

KeyboardInterrupt: 